In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Environment ready and libraries loaded!")


✅ Environment ready and libraries loaded!


In [5]:
import pandas as pd
import os

# Directorio de datos, esto indica donde estan guardado los archivos CSV.
data_path = "../data/"

import os

# os.listdir(path): devuelve una lista con los nombres de archivos y carpetas dentro del directorio indicado.
# En este caso, listamos los archivos dentro de la carpeta "../data/"
files = [f for f in os.listdir("../data/") if f.endswith(".csv")]

# .endswith(".csv"): método de los strings que verifica si el texto termina con un patrón determinado.
# Aquí se usa para filtrar solo los archivos CSV.
print("Archivos CSV encontrados:", files)


# --------------------------------------------------------------
# 🧩 ETAPA: Lectura y combinación de archivos CSV (EDA)
# Objetivo:
#   - Leer múltiples archivos (uno por año) desde la carpeta "data"
#   - Agregar una columna 'Year' con el año correspondiente
#   - Combinar todos los DataFrames en uno solo para el análisis
# --------------------------------------------------------------

# Lista vacía donde guardaremos temporalmente los DataFrames individuales
df_list = []

# Recorremos cada archivo CSV encontrado en la lista "files"
for file in files:
    
    # 1️⃣ Extraer el año del nombre del archivo (antes del punto ".")
    # Ejemplo: "2015.csv" → "2015"
    year = file.split(".")[0]
    
    # 2️⃣ Leer el archivo CSV y cargarlo como un DataFrame temporal
    # os.path.join() asegura que la ruta funcione en Windows, Mac o Linux
    temp = pd.read_csv(os.path.join(data_path, file))
    
    # 3️⃣ Agregar una columna nueva llamada "Year" con el año correspondiente
    temp["Year"] = year
    
    # 4️⃣ Guardar este DataFrame temporal en la lista
    df_list.append(temp)

# --------------------------------------------------------------
# 🔹 Unir todos los DataFrames en uno solo
# ignore_index=True reinicia los índices para evitar duplicados
# --------------------------------------------------------------
df = pd.concat(df_list, ignore_index=True)

# Mostrar las primeras filas del DataFrame combinado
df.head()



Archivos CSV encontrados: ['2015.csv', '2016.csv', '2017.csv', '2018.csv', '2019.csv']


,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),...,Trust..Government.Corruption.,Dystopia.Residual,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Perceptions of corruption
0,Switzerland,Western Europe,1.0,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Iceland,Western Europe,2.0,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Denmark,Western Europe,3.0,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Norway,Western Europe,4.0,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Canada,North America,5.0,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Ver el tamaño del dataset (filas y columnas)
print("Shape:", df.shape)

# Ver los nombres de las columnas
print("\nColumnas:")
print(df.columns.tolist())

# Ver las primeras filas
df.head()


Shape: (782, 31)

Columnas:
['Country', 'Region', 'Happiness Rank', 'Happiness Score', 'Standard Error', 'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual', 'Year', 'Lower Confidence Interval', 'Upper Confidence Interval', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high', 'Whisker.low', 'Economy..GDP.per.Capita.', 'Health..Life.Expectancy.', 'Trust..Government.Corruption.', 'Dystopia.Residual', 'Overall rank', 'Country or region', 'Score', 'GDP per capita', 'Social support', 'Healthy life expectancy', 'Freedom to make life choices', 'Perceptions of corruption']


,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),...,Trust..Government.Corruption.,Dystopia.Residual,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Perceptions of corruption
0,Switzerland,Western Europe,1.0,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Iceland,Western Europe,2.0,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Denmark,Western Europe,3.0,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Norway,Western Europe,4.0,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Canada,North America,5.0,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# ======================================================

# Exploracion inicial del dataset.

# =======================================================


# Ver informacion general (tipos de datos, valores nulos, etc.)

print("Informacion general del dataframe:")
df.info()

# Contar valores nulos.

print("\n Valores nulos por columnas:")
print(df.isnull().sum())


# Revisar dupplicados
print("\n Numero de filas duplicadas:", df.duplicated().sum())

# Resumen estadistico  (solo numeros)
print("\n Resumen estadistico:")
display(df.describe())


Informacion general del dataframe:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Country                        470 non-null    object 
 1   Region                         315 non-null    object 
 2   Happiness Rank                 315 non-null    float64
 3   Happiness Score                315 non-null    float64
 4   Standard Error                 158 non-null    float64
 5   Economy (GDP per Capita)       315 non-null    float64
 6   Family                         470 non-null    float64
 7   Health (Life Expectancy)       315 non-null    float64
 8   Freedom                        470 non-null    float64
 9   Trust (Government Corruption)  315 non-null    float64
 10  Generosity                     782 non-null    float64
 11  Dystopia Residual              315 non-null    float64
 12  Year           

,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual,...,Health..Life.Expectancy.,Trust..Government.Corruption.,Dystopia.Residual,Overall rank,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Perceptions of corruption
count,315.000000,315.000000,158.000000,315.000000,470.000000,315.000000,470.000000,315.000000,782.000000,315.000000,...,155.000000,155.000000,155.000000,312.000000,312.000000,312.000000,312.000000,312.000000,312.000000,311.000000
mean,79.238095,5.378949,0.047885,0.899837,0.990347,0.594054,0.402828,0.140532,0.218576,2.212032,...,0.551341,0.123120,1.850238,78.500000,5.391506,0.898298,1.211026,0.661295,0.423538,0.111299
std,45.538922,1.141531,0.017146,0.410780,0.318707,0.240790,0.150356,0.115490,0.122321,0.558728,...,0.237073,0.101661,0.500028,45.104737,1.114631,0.394592,0.300310,0.252724,0.156024,0.095365
min,1.000000,2.839000,0.018480,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.328580,...,0.000000,0.000000,0.377914,1.000000,2.853000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,4.510000,0.037268,0.594900,0.793000,0.419645,0.297615,0.061315,0.130000,1.884135,...,0.369866,0.057271,1.591291,39.750000,4.514250,0.609500,1.055750,0.488250,0.322500,0.050000
50%,79.000000,5.286000,0.043940,0.973060,1.025665,0.640450,0.418347,0.106130,0.201982,2.211260,...,0.606042,0.089848,1.832910,78.500000,5.379500,0.960000,1.265500,0.699000,0.449500,0.082000
75%,118.500000,6.269000,0.052300,1.229000,1.228745,0.787640,0.516850,0.178610,0.278832,2.563470,...,0.723008,0.153296,2.144654,117.250000,6.176000,1.219500,1.457500,0.858000,0.540250,0.140500
max,158.000000,7.587000,0.136930,1.824270,1.610574,1.025250,0.669730,0.551910,0.838075,3.837720,...,0.949492,0.464308,3.117485,156.000000,7.769000,2.096000,1.644000,1.141000,0.724000,0.457000


In [12]:
# Listado de columas en forma de lista.


print(df.columns.tolist())


['Country', 'Region', 'Happiness Rank', 'Happiness Score', 'Standard Error', 'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual', 'Year', 'Lower Confidence Interval', 'Upper Confidence Interval', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high', 'Whisker.low', 'Economy..GDP.per.Capita.', 'Health..Life.Expectancy.', 'Trust..Government.Corruption.', 'Dystopia.Residual', 'Overall rank', 'Country or region', 'Score', 'GDP per capita', 'Social support', 'Healthy life expectancy', 'Freedom to make life choices', 'Perceptions of corruption']


In [13]:
# Muestra una tabla vertical.

for i, col in enumerate(df.columns):
    print(f"{i + 1:02d}. {col}")

01. Country
02. Region
03. Happiness Rank
04. Happiness Score
05. Standard Error
06. Economy (GDP per Capita)
07. Family
08. Health (Life Expectancy)
09. Freedom
10. Trust (Government Corruption)
11. Generosity
12. Dystopia Residual
13. Year
14. Lower Confidence Interval
15. Upper Confidence Interval
16. Happiness.Rank
17. Happiness.Score
18. Whisker.high
19. Whisker.low
20. Economy..GDP.per.Capita.
21. Health..Life.Expectancy.
22. Trust..Government.Corruption.
23. Dystopia.Residual
24. Overall rank
25. Country or region
26. Score
27. GDP per capita
28. Social support
29. Healthy life expectancy
30. Freedom to make life choices
31. Perceptions of corruption


In [14]:
# Las ordena alfabeticamente.

sorted(df.columns)


['Country',
 'Country or region',
 'Dystopia Residual',
 'Dystopia.Residual',
 'Economy (GDP per Capita)',
 'Economy..GDP.per.Capita.',
 'Family',
 'Freedom',
 'Freedom to make life choices',
 'GDP per capita',
 'Generosity',
 'Happiness Rank',
 'Happiness Score',
 'Happiness.Rank',
 'Happiness.Score',
 'Health (Life Expectancy)',
 'Health..Life.Expectancy.',
 'Healthy life expectancy',
 'Lower Confidence Interval',
 'Overall rank',
 'Perceptions of corruption',
 'Region',
 'Score',
 'Social support',
 'Standard Error',
 'Trust (Government Corruption)',
 'Trust..Government.Corruption.',
 'Upper Confidence Interval',
 'Whisker.high',
 'Whisker.low',
 'Year']

In [17]:
# =================================

# Limpieza de nombres de columnas

# =================================

# Para ver columnas unicas.

print("Antes de limpiar:")
print(df.columns.tolist())

# Normalizar nombres: minusculas, sin espacios ni puntos.

df.columns = (
    df.columns
    .str.strip()   # quitar espacios, al inicio o al final.
    .str.lower()    # pasar a minusculas. Convierte todos los caracteres a minusculas.
    .str.replace(' ', '_')   # reemplazar espacios vacios  por _ .  Ejemplo: "Happiness Score" → "happiness_score"
    .str.replace('.','_') #reemplazar puntos .  por _ . Ejemplo: "Happiness.Score" → "happiness_score"
)

print("\n Despues de limpiar:")
print(df.columns.tolist())




Antes de limpiar:
['Country', 'Region', 'Happiness Rank', 'Happiness Score', 'Standard Error', 'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual', 'Year', 'Lower Confidence Interval', 'Upper Confidence Interval', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high', 'Whisker.low', 'Economy..GDP.per.Capita.', 'Health..Life.Expectancy.', 'Trust..Government.Corruption.', 'Dystopia.Residual', 'Overall rank', 'Country or region', 'Score', 'GDP per capita', 'Social support', 'Healthy life expectancy', 'Freedom to make life choices', 'Perceptions of corruption']

 Despues de limpiar:
['country', 'region', 'happiness_rank', 'happiness_score', 'standard_error', 'economy_(gdp_per_capita)', 'family', 'health_(life_expectancy)', 'freedom', 'trust_(government_corruption)', 'generosity', 'dystopia_residual', 'year', 'lower_confidence_interval', 'upper_confidence_interval', 'happiness_rank', 'happiness_score',

In [20]:
## Revision de similitud de columnas

# Nivel 1: Revision manual basica.

# Ver todas las columnas (ya nombres limpios)

for i, col in enumerate(df.columns):
    print(f"{i + 1:02d}. {col}")

#Contar cuantas columnas tiene el dataframe.

print("\n Total de columnas:", len(df.columns))

01. country
02. region
03. happiness_rank
04. happiness_score
05. standard_error
06. economy_(gdp_per_capita)
07. family
08. health_(life_expectancy)
09. freedom
10. trust_(government_corruption)
11. generosity
12. dystopia_residual
13. year
14. lower_confidence_interval
15. upper_confidence_interval
16. happiness_rank
17. happiness_score
18. whisker_high
19. whisker_low
20. economy__gdp_per_capita_
21. health__life_expectancy_
22. trust__government_corruption_
23. dystopia_residual
24. overall_rank
25. country_or_region
26. score
27. gdp_per_capita
28. social_support
29. healthy_life_expectancy
30. freedom_to_make_life_choices
31. perceptions_of_corruption

 Total de columnas: 31


In [27]:
# Nivel 2: Comparacion automatica de nombres similares. El objetivo es detectar columnas de nombres parecidos. (posibles duplicados)

import difflib   # Libreria estandar de Python para comparar similitudes de texto.

cols = df.columns.tolist()  # Convertimos el listado de columnas del Dataframe en una lista normal.

# Mensaje informativo para saber que hace la celda.

print("Posibles columnas duplicadas o equivalentes:\n")


#Bucle principal: recorre cada columna una por una

for i, col1 in enumerate(cols):
    #Bucle interno: compara col1 con las columnas siguientes (evita repetir comparaciones)
    for col2 in cols[i + 1:]:

        #Calcula la similitud entre los nombres de las columnas
        #El valor va entre 0 (totalmente diferentes) y 1 (identicos)
        similarity = difflib.SequenceMatcher(None, col1, col2).ratio()

        # Si la similitud es mayor al 80% y no son exactamente iguales.
        if similarity > 0.7 and col1 != col2:

                # Imprime el par de columnas similares y su nivel de similitud. Por ejemplo 0.93.
                print(f"{col1:35} ↔ {col2:35}  ({similarity:.2f})")








Posibles columnas duplicadas o equivalentes:

happiness_rank                      ↔ happiness_score                      (0.76)
happiness_rank                      ↔ happiness_score                      (0.76)
happiness_score                     ↔ happiness_rank                       (0.76)
economy_(gdp_per_capita)            ↔ economy__gdp_per_capita_             (0.92)
economy_(gdp_per_capita)            ↔ gdp_per_capita                       (0.74)
health_(life_expectancy)            ↔ health__life_expectancy_             (0.92)
health_(life_expectancy)            ↔ healthy_life_expectancy              (0.94)
trust_(government_corruption)       ↔ trust__government_corruption_        (0.93)
lower_confidence_interval           ↔ upper_confidence_interval            (0.88)
happiness_rank                      ↔ happiness_score                      (0.76)
economy__gdp_per_capita_            ↔ gdp_per_capita                       (0.74)
health__life_expectancy_            ↔ healthy_life_e